In terminal, start the container as:<br>
`ml apptainer`<br>
`apptainer run --fakeroot --nv -e -B /scratch/general/vast/u0101881/nvs/.cache/nim/models:/home/nvs/.cache -B /scratch/general/vast/u0101881/nim/proteinmpnn/workspace:/opt/nim/workspace --env NIM_CACHE_PATH=/home/nvs/.cache/nim/models,NIM_HTTP_API_PORT=8083 /uufs/chpc.utah.edu/common/home/u0101881/nvidia/nim/proteinmpnn_1.0.sif`<br>
Once it starts, it'll show<br>
```
INFO:uvicorn.error:Application startup complete.
INFO:uvicorn.error:Uvicorn running on http://0.0.0.0:8083 (Press CTRL+C to quit)
```

In [1]:
!curl localhost:8083/v1/health/ready

{"status":"ready"}

In [2]:
import requests
import os
import json
from pathlib import Path

In [3]:
def get_reduced_pdb():
    pdb = Path("1R42.pdb")
    if not pdb.exists():
        pdb.write_text(requests.get(f"https://files.rcsb.org/download/{pdb}").text)
    lines = filter(lambda line: line.startswith("ATOM"), pdb.read_text().split("\n"))
    return "\n".join(list(lines)[:200])


In [4]:
r = requests.post(
    url="http://localhost:8083/biology/ipd/proteinmpnn/predict",
    json={
        "input_pdb": get_reduced_pdb(),
        "ca_only": False,
        "use_soluble_model": False,
        "sampling_temp": [0.1],
    },
)
print(r, "Saving to output.fa:\n", r.text[:200], "...")
Path("output.fa").write_text(json.loads(r.text)["mfasta"])


<Response [200]> Saving to output.fa:
 {"mfasta":">input, score=2.6678, global_score=2.6678, fixed_chains=[], designed_chains=['A'], model_name=v_48_002, git_hash=unknown, seed=102\nSTIEEQAKTFLDKFNHEAEDLFYQ\n>T=0.1, sample=1, score=1.2692, ...


254